In [ ]:
import pandas as pd
import numpy as np
import numpy as np
np.float = float
np.bool = bool
import matplotlib.pyplot as plt
import seaborn as sns
from emukit.core import ParameterSpace, ContinuousParameter, DiscreteParameter
from emukit.core.initial_designs.latin_design import LatinDesign
from emukit.core.initial_designs.random_design import RandomDesign
#from chimera import Chimera
from emukit.core.loop.user_function import UserFunctionWrapper
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold, cross_val_score, cross_val_predict
import plotly.express as px
import plotly.graph_objects as go
from numpy.linalg import norm
import os
import scipy.stats
%config InlineBackend.figure_format = 'retina'
%matplotlib inline
plt.rcParams['font.sans-serif'] = ["Arial"]

In [ ]:
TEP_min, TEP_max, TEP_step = [500, 650, 10] ## Unit: %, 11 steps
TEP_var = np.arange(TEP_min, TEP_max+TEP_step*0.1, TEP_step)
TEP_num = len(TEP_var)

MACl_min, MACl_max, MACl_step = [10, 40, 5] ## Unit: %, 11 steps
MACl_var = np.arange(MACl_min, MACl_max+MACl_step*0.1, MACl_step)
MACl_num = len(MACl_var)

TwoMe_min, TwoMe_max, TwoMe_step = [0, 50, 5] ## Unit: uL
TwoMe_var = np.arange(TwoMe_min, TwoMe_max+TwoMe_step*0.1, TwoMe_step) 
TwoMe_num = len(TwoMe_var)

NMP_min, NMP_max, NMP_step = [10, 80, 5] ## Unit: %, 21 steps
NMP_var = np.arange(NMP_min, NMP_max+NMP_step*0.1, NMP_step)
NMP_num = len(NMP_var)




var_array = [TEP_var, MACl_var, 
             NMP_var,TwoMe_var]
x_labels = ['TEP (μl)',
            'MACl (%)',
            'NMP (μl)',
           '2-Me (μL)']  

In [ ]:
def x_normalizer(X, var_array = var_array):
    
    def max_min_scaler(x, x_max, x_min):
        return (x-x_min)/(x_max-x_min)
    x_norm = []
    for x in (X):
           x_norm.append([max_min_scaler(x[i], 
                         max(var_array[i]), 
                         min(var_array[i])) for i in range(len(x))])
            
    return np.array(x_norm)

def x_denormalizer(x_norm, var_array = var_array):
    
    def max_min_rescaler(x, x_max, x_min):
        return x*(x_max-x_min)+x_min
    x_original = []
    for x in (x_norm):
           x_original.append([max_min_rescaler(x[i], 
                              max(var_array[i]), 
                              min(var_array[i])) for i in range(len(x))])

    return np.array(x_original)

def get_closest_value(given_value, array_list):
    absolute_difference_function = lambda list_value : abs(list_value - given_value)
    closest_value = min(array_list, key=absolute_difference_function)
    return closest_value
    
def get_closest_array(suggested_x, var_list):
    modified_array = []
    for x in suggested_x:
        modified_array.append([get_closest_value(x[i], var_list[i]) for i in range(len(x))])
    return np.array(modified_array)

In [ ]:
parameter_space2 = ParameterSpace([ContinuousParameter('x1', 0-1/(TEP_num-1)/2,  1+1/(TEP_num-1)/2),
                                  ContinuousParameter('x2', 0-1/(MACl_num-1)/2, 1+1/(MACl_num-1)/2),
                                  ContinuousParameter('x3', 0-1/(NMP_num-1)/2,  1+1/(NMP_num-1)/2),
                                  ContinuousParameter('x4', 0-1/(TwoMe_num-1)/2, 1+1/(TwoMe_num-1)/2)])

In [ ]:
df_exp = pd.read_csv('solibility.csv',encoding = 'gb2312')
df_exp

In [ ]:

x2=df_exp.iloc[:,0:4].values#process conditions

x2=x_normalizer(x2)
y2=np.transpose([df_exp.iloc[:,-1].values])#C Solution

In [ ]:
import GPy
from GPy.models import GPRegression
from emukit.model_wrappers import GPyModelWrapper
X2, Y2 = [x2, y2]
input_dim2 = len(X2[0])
print(len(X2[0]))
ker2 = GPy.kern.RBF(input_dim = input_dim2, ARD =True)
ker2.lengthscale.constrain_bounded(1e-1, 1)
ker2.variance.constrain_bounded(1e-1, 1e3)
model2_gpy = GPRegression(X2, -Y2, ker2)
model2_gpy.Gaussian_noise.variance =0.05**2
model2_gpy.Gaussian_noise.variance.fix()
model2_gpy.randomize()
model2_gpy.optimize_restarts(num_restarts=10,verbose =True, messages=False)
objective_model2 = GPyModelWrapper(model2_gpy)
print(objective_model2.model.kern.lengthscale)
print(objective_model2.model.kern.variance)
f_obj2 =  objective_model2.model.predict
y2_pred, y2_uncer = f_obj2(X2)
y2_pred = -y2_pred[:,-1]
y2_uncer = np.sqrt(y2_uncer[:,-1])

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from matplotlib.font_manager import FontProperties


font_arial = FontProperties(family='Arial')
fs = 20
lims = (-0.1, 1.1)

fig, ax = plt.subplots(figsize=(5.5, 5.5))  

# Spray Probability
ax.scatter(Y2[:, -1], y2_pred, alpha=0.5, c='navy', edgecolor='navy')
ax.errorbar(Y2[:, -1], y2_pred, yerr=y2_uncer, ms=0, ls='', capsize=2, alpha=0.6, color='gray', zorder=0)
ax.plot(lims, lims, color='black', linestyle=(0, (3, 3)), linewidth=2, alpha=0.8, zorder=0)
ax.set_xlim(lims)
ax.set_ylim(lims)
ax.set_xlabel('Ground Truth Solubility', fontsize=fs, fontproperties=font_arial, labelpad=10)
ax.set_ylabel('Predicted Solubility', fontsize=fs, fontproperties=font_arial, labelpad=10)
ax.tick_params(direction='in', length=5, width=1, labelsize=fs, grid_alpha=0.5)
ax.grid(True, linestyle='-.')
ax.tick_params(axis='x', pad=10)  
ax.tick_params(axis='y', pad=10) 
plt.tight_layout()


plt.savefig('solubility.png', dpi=600, bbox_inches='tight', facecolor='white', transparent=False)

plt.show()


In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from scipy.stats import spearmanr
import numpy as np


rmse_value = np.sqrt(mean_squared_error(Y2[:, -1], y2_pred))
mae_value = mean_absolute_error(Y2[:, -1], y2_pred)
spearman_value = spearmanr(Y2[:, -1], y2_pred)[0]
rsquared_value = r2_score(Y2[:, -1], y2_pred)


print('MAE:', np.round(mae_value, 4),
      'RMSE:', np.round(rmse_value, 4),
      'Spearman:', np.round(spearman_value, 4),
      'R² score:', np.round(rsquared_value, 4))

In [ ]:
design = RandomDesign(parameter_space1)
x1_sampled = design.get_samples(200)
n_steps =21
x1x2 = []
for x1 in np.linspace(0, 1, n_steps):
    for x2 in np.linspace(0, 1, n_steps):
        x_temp1 = np.copy(x1_sampled)
        x_temp1[:,0] = x1
        x_temp1[:,1] = x2
        x1_org = x_denormalizer(x_temp1)[0,0]
        x2_org = x_denormalizer(x_temp1)[0,1]
        x1x2.append([x1_org, x2_org])

x1 = np.array(x1x2, dtype=object)[:,0].reshape(n_steps, n_steps)
x2 = np.array(x1x2, dtype=object)[:,1].reshape(n_steps, n_steps)

In [ ]:
design = RandomDesign(parameter_space2)
x_sampled = design.get_samples(200)
x_sampled = x_sampled
input_dim = 4
for i in range(input_dim):
    for j in range(input_dim-i-1):
        ind1 = i
        ind2 = j+i+1
        n_steps =21
        x1x2y_pred, x1x2y_uncer =[[],[]]
        for x1 in np.linspace(0, 1, n_steps):
            for x2 in np.linspace(0, 1, n_steps):
                x_temp = np.copy(x_sampled)
                x_temp[:,ind1] = x1
                x_temp[:,ind2] = x2
                y_pred, y_uncer = f_obj2(x_temp)
                y2 = -y_pred
                cdf = 1-scipy.stats.norm.cdf(0.5, y2, y_uncer)
                x1_org = x_denormalizer(x_temp)[0,ind1]
                x2_org = x_denormalizer(x_temp)[0,ind2]
                x1x2y_pred.append([x1_org, x2_org, np.max(y2), np.mean(y2), np.min(y2),np.max(cdf), np.mean(cdf), np.min(cdf)])
                x1x2y_uncer.append([x1_org, x2_org, np.max(np.sqrt(y_uncer)), np.mean(np.sqrt(y_uncer)), np.min(np.sqrt(y_uncer))])
        
        x1 = np.array(x1x2y_pred, dtype=object)[:,0].reshape(n_steps, n_steps)
        x2 = np.array(x1x2y_pred, dtype=object)[:,1].reshape(n_steps, n_steps)
            
        y_max2 = np.array(x1x2y_pred, dtype=object)[:,2].reshape(n_steps, n_steps)
        y_mean2 = np.array(x1x2y_pred, dtype=object)[:,3].reshape(n_steps, n_steps)
        y_min2 = np.array(x1x2y_pred, dtype=object)[:,4].reshape(n_steps, n_steps)
        cdf_max = np.array(x1x2y_pred, dtype=object)[:,5].reshape(n_steps, n_steps)
        cdf_mean = np.array(x1x2y_pred, dtype=object)[:,6].reshape(n_steps, n_steps)
        cdf_min = np.array(x1x2y_pred, dtype=object)[:,7].reshape(n_steps, n_steps)
        
        y_uncer_max = np.array(x1x2y_uncer, dtype=object)[:,2].reshape(n_steps, n_steps)
        y_uncer_mean = np.array(x1x2y_uncer, dtype=object)[:,3].reshape(n_steps, n_steps)
        y_uncer_min = np.array(x1x2y_uncer, dtype=object)[:,4].reshape(n_steps, n_steps)

        fs = 24
        title_pad = 16
        
        fig,axes = plt.subplots(1, 3, figsize=(17, 4), sharey = False, sharex = False)
        for ax, y in zip(axes,
                           [y_max2, y_mean2, y_min2]):
            c_plt1 = ax.contourf(x1, x2, y, levels = np.arange(10,24)*0.04, cmap='plasma',extend='both')
            #extend='both'
#             std = ax.contour(x1, x2, y,[0.5],cmap='coolwarm')
            cbar = fig.colorbar(c_plt1, ax= ax)
            cbar.ax.tick_params(labelsize=fs*0.7)
            ax.scatter(x_denormalizer(X2)[:, ind1], 
                       x_denormalizer(X2)[:, ind2], 
                       s = 50, facecolors='none', alpha = 0.9, edgecolor = 'red')
            
            ax.set_xlabel(str(x_labels[ind1]),fontsize =  fs,labelpad=10)
            ax.set_ylabel(str(x_labels[ind2]),fontsize =  fs,labelpad=10)
            
            x1_delta = (np.max(x1)-np.min(x1))*0.05
            x2_delta = (np.max(x2)-np.min(x2))*0.05
            ax.set_xlim(np.min(x1)-x1_delta, np.max(x1)+x1_delta)
            ax.set_ylim(np.min(x2)-x2_delta, np.max(x2)+x2_delta)
            
            ax.tick_params(direction='in', length=5, width=1, labelsize = 20)#, grid_alpha = 0.5
            if ind1==0:
                ax.set_xticks([500, 550, 600, 650])
            if ind1==1:
                ax.set_xticks([10, 20, 30, 40])
            if ind1==2:
                ax.set_yticks([10, 25, 40, 55, 70, 85])
            if ind1==3:#Q
                  ax.set_yticks([0, 10, 20, 30,40,50 ])
                
        axes[0].set_title('Solubility max', pad = title_pad,fontsize =  fs)
        axes[1].set_title('Solubility mean', pad = title_pad,fontsize =  fs)
        axes[2].set_title('Solubility min', pad = title_pad,fontsize =  fs)

        plt.subplots_adjust(wspace = 0.3)
        plt.show()
        
        fig,axes = plt.subplots(1, 3, figsize=(17, 4), sharey = False, sharex = False)
        for ax, y in zip(axes,
                           [cdf_max,cdf_mean,cdf_min]):
            c_plt2 = ax.contourf(x1, x2, y, levels = np.arange(3,10)*0.1, cmap='plasma',extend='both')
#             std = ax.contour(x1, x2, y,[0.5],cmap='coolwarm')
            cbar = fig.colorbar(c_plt2, ax= ax)
            cbar.ax.tick_params(labelsize=fs*0.8)
            ax.scatter(x_denormalizer(X2)[:, ind1], 
                       x_denormalizer(X2)[:, ind2], 
                       s = 50, facecolors='none', alpha = 0.9, edgecolor = 'red')
            
            ax.set_xlabel(str(x_labels[ind1]),fontsize =  fs,labelpad=10)
            ax.set_ylabel(str(x_labels[ind2]),fontsize =  fs,labelpad=10)
            
            x1_delta = (np.max(x1)-np.min(x1))*0.05
            x2_delta = (np.max(x2)-np.min(x2))*0.05
            ax.set_xlim(np.min(x1)-x1_delta, np.max(x1)+x1_delta)
            ax.set_ylim(np.min(x2)-x2_delta, np.max(x2)+x2_delta)
            ax.tick_params(direction='in', length=5, width=1, labelsize = 20)#, grid_alpha = 0.5
            if ind1==0:
                ax.set_xticks([500, 550, 600, 650])
            if ind1==1:
                ax.set_xticks([10, 20, 30, 40])
            if ind1==2:
                ax.set_yticks([10, 25, 40, 55, 70, 85])
            if ind1==3:#Q
                  ax.set_yticks([0, 10, 20, 30,40,50 ])
                
#             if ind1==3:#Q
#                 ax.set_yticks([0.4, 0.8, 1.2, 1.6])
                
        axes[0].set_title('Solubility cdf max', pad = title_pad,fontsize =  fs)
        axes[1].set_title('Solubility cdf mean', pad = title_pad,fontsize =  fs)
        axes[2].set_title('Solubility cdf min', pad = title_pad,fontsize =  fs)

        plt.subplots_adjust(wspace = 0.3)
        plt.show()
        
        fig,axes = plt.subplots(1, 1, figsize=(5, 4), sharey =False, sharex = False) 
        colorbar_offset = [0.4]
        c_plt3 = axes.contourf(x1, x2, y_uncer_max, levels = np.arange(6)*0.05+colorbar_offset, cmap='plasma',extend='both')
        cbar = fig.colorbar(c_plt3, ax = axes)
        axes.set_xlabel(str(x_labels[ind1]),fontsize =  fs,labelpad=10)
        axes.set_ylabel(str(x_labels[ind2]),fontsize =  fs,labelpad=10)

        x1_delta = (np.max(x1)-np.min(x1))*0.05
        x2_delta = (np.max(x2)-np.min(x2))*0.05
        axes.set_xlim(np.min(x1)-x1_delta, np.max(x1)+x1_delta)
        axes.set_ylim(np.min(x2)-x2_delta, np.max(x2)+x2_delta)
        axes.tick_params(direction='in', length=5, width=1, labelsize = 20)#, grid_alpha = 0.5
        if ind1==0:
            ax.set_xticks([500, 550, 600, 650])
        if ind1==1:
            ax.set_xticks([10, 20, 30, 40])
        if ind1==2:
            ax.set_yticks([10, 25, 40, 55, 70, 85])
        if ind1==3:#Q
                  ax.set_yticks([0, 10, 20, 30,40,50 ])
                
#         if ind1==3:#Q
#             axes.set_yticks([0.4, 0.8, 1.2, 1.6])
            
        axes.scatter(x_denormalizer(X2)[:, ind1], 
                       x_denormalizer(X2)[:, ind2], 
                       s = 50, facecolors='none', alpha = 0.9, edgecolor = 'red')
#         ax.scatter(x_2[:, 0], 
#                        x_2[:, 1], 
#                        s = 50, facecolors='none', alpha = 0.9, edgecolor = 'green')
        axes.set_title('Solubility variance max', pad = title_pad,fontsize =  fs)